## Encoding categorial features
Categorical features are non-numeric features with a limited amount of possible options. 

Before feeding them into most machine learning algorithms, we must convert them into numerical features using either:
- **Label encoding:** Each unique category is assigned into a numerical value: 0, 1, 2, 3 ...
- **One-hot encoding:** A new binary feature is created for each category. (T.ex. för 1, 2 och 3 klass så görs tre kolumner och i varje kolumn skriver 0 eller 1). Anledningen att man vill göra så här och inte ha 0-6, efter maskininlärningsalgoritmer hade räknat i viss ordning, funkar inte alltid; t.ex. länder som inte har inbördes rangordning. Då kommer man runt det med olika kolumner istället.
Detta eftersom maskininlärningsalgoritmer jobbar på siffror och inte strängar

In [2]:
import pandas as pd
import seaborn as sns

In [3]:
titanic = sns.load_dataset("titanic")
titanic.head(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False


In [4]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# Vill få ut kolumner för categoricals

# titanic.columns # lista med alla kolumner, denna gör man comprehension på nedan
# Seaborn är byggt på panda, så därför skapar den tabeller i pandas datatyper

cat_columns = [col for col in titanic.columns if titanic[col].dtype in ["object", "category"]]

cat_columns




['sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive']

In [6]:
# Skapar en dataframe med enbart de kategoriernas kolumner 
# eftersom cat_columns är en lista med de kolumner jag vill ha så blir det 
# kolumn indexing med flera kolumner 

df_categories = titanic[cat_columns]

df_categories.head(1)

,sex,embarked,class,who,deck,embark_town,alive
0,male,S,Third,man,NaN,Southampton,no


In [7]:
# Loopar igenom kolumnerna för att få se alla parametrar som finns för de
for cat in cat_columns:
    print(f"{cat}: {df_categories[cat].unique()}")    # Får ut alla unika parametrar för varje kolumn

sex: ['male' 'female']
embarked: ['S' 'C' 'Q' nan]
class: ['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']
who: ['man' 'woman' 'child']
deck: [NaN, 'C', 'E', 'G', 'D', 'A', 'B', 'F']
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']
embark_town: ['Southampton' 'Cherbourg' 'Queenstown' nan]
alive: ['no' 'yes']


### Label encoding 
Label encoding maps each category into a numerical value.

Use label encoding if:
- Categories have a natural order
- There are only 2 categories
- If using One-hot encoding leads to a large number of features (columns)

In [8]:
#  Label encoding - Manual mapping
# Nu behöver dessa categoricals göras till numericals enligt de angivna metoderna

# embarked feature can have values: "S", "C" and "Q"
titanic["embarked"].map({"S": 0, "C": 1, "Q": 2})         # Bara uttryck för output, ej sparats i kolumnen, (se längst ned för hur man sparar)

# map() tar en dictionary och säger hur värdet ska mappas från ena till andra
# Nu har vi mappat på ett manuellt enkelt sätt med map.

# Om vi vill spara i kolumnen
titanic["embarked"] = titanic["embarked"].map({"S": 0, "C": 1, "Q": 2})        

#### Auto-mapping using Label Encoder from Scikit-learn

In [9]:
# Use: pipenv install scikit-learn to install package

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()      # importerar denna klass och gör en instans av den

titanic["deck"]      # gör man såhär så ser man lite från början och slutet.


0      NaN
1        C
2      NaN
3        C
4      NaN
      ... 
886    NaN
887      B
888    NaN
889      C
890    NaN
Name: deck, Length: 891, dtype: category
Categories (7, object): ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [13]:
titanic["deck_no"] = le.fit_transform(titanic["deck"]) # skapar ny kolumn där denna automatiskt lablar

titanic[["deck", "deck_no"]]   
# Här ser vi att den numrerat 0-6 och NaN 7.
# Man hade helst städat bort NaN först så hade det inte blivit så
# Kontrollera i efterhand att det blev rätt med sorteringsordning o.s.v.

# När jag ser att det ser ut som jag vill kan jag ändra den kolumnen
titanic["deck"] = titanic["deck_no"]

     deck  deck_no
0       7        7
1       2        2
2       7        7
3       2        2
4       7        7
..    ...      ...
886     7        7
887     1        1
888     7        7
889     2        2
890     7        7

[891 rows x 2 columns]


### One-hot encoding
In one-hot encoding, a new binary feature is created for each category, and the value of that feature is set to 1 if the observation belongs to that category, and 0 otherwise. 

Use one-hot encoding if:
- Categories have no natural order.
- Number of categories are small (but not 2)

In [11]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,deck_no
0,0,3,male,22.0,1,0,7.2500,0.0,Third,man,True,7,Southampton,no,False,7
1,1,1,female,38.0,1,0,71.2833,1.0,First,woman,False,2,Cherbourg,yes,False,2
2,1,3,female,26.0,0,0,7.9250,0.0,Third,woman,False,7,Southampton,yes,True,7
3,1,1,female,35.0,1,0,53.1000,0.0,First,woman,False,2,Southampton,yes,False,2
4,0,3,male,35.0,0,0,8.0500,0.0,Third,man,True,7,Southampton,no,True,7


In [12]:
# Get gummies metod i pandas for one-hot encoding

pd.get_dummies(data = titanic, columns = ["who"])


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,adult_male,deck,embark_town,alive,alone,deck_no,who_child,who_man,who_woman
0,0,3,male,22.0,1,0,7.2500,0.0,Third,True,7,Southampton,no,False,7,False,True,False
1,1,1,female,38.0,1,0,71.2833,1.0,First,False,2,Cherbourg,yes,False,2,False,False,True
2,1,3,female,26.0,0,0,7.9250,0.0,Third,False,7,Southampton,yes,True,7,False,False,True
3,1,1,female,35.0,1,0,53.1000,0.0,First,False,2,Southampton,yes,False,2,False,False,True
4,0,3,male,35.0,0,0,8.0500,0.0,Third,True,7,Southampton,no,True,7,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,0.0,Second,True,7,Southampton,no,True,7,False,True,False
887,1,1,female,19.0,0,0,30.0000,0.0,First,False,1,Southampton,yes,True,1,False,False,True
888,0,3,female,NaN,1,2,23.4500,0.0,Third,False,7,Southampton,no,False,7,False,False,True
889,1,1,male,26.0,0,0,30.0000,1.0,First,True,2,Cherbourg,yes,True,2,False,True,False


In [13]:
# Om vi vill få det i originalet:
titanic = pd.get_dummies(data = titanic, columns = ["who"])

### Auto-map remaining non-numerical features

In [15]:
# Inkluderar alla kvarvarande nu, även bools (True, False som blir 1 eller 0)
cat_columns = [col for col in titanic.columns if titanic[col].dtype in ["object", "category", "bool"]]

cat_columns

['sex', 'class', 'who', 'adult_male', 'embark_town', 'alive', 'alone']

In [16]:
for column in cat_columns:
    titanic[column] = le.fit_transform(titanic[column])

In [19]:
# Nu tittar vi på dataframe:t igen och allt är siffror
# Så som maskininlärningsalgoritmer vill ha det. 
# Kolla i dokumentationen för att se hur man sparar originaldatan för
# att jämföra och minnas vad siffrorna betyder.

titanic.head

#Sparar detta dataset för att ladda in i L010_Correlation_analysis.ipynb där det måste vara numerics för att köra pandas corr() metod

titanic.to_json("../Data/titanic_encode.json", orient = "records")

titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,deck_no
0,0,3,1,22.0,1,0,7.2500,0.0,2,1,1,7,2,0,0,7
1,1,1,0,38.0,1,0,71.2833,1.0,0,2,0,2,0,1,0,2
2,1,3,0,26.0,0,0,7.9250,0.0,2,2,0,7,2,1,1,7
3,1,1,0,35.0,1,0,53.1000,0.0,0,2,0,2,2,1,0,2
4,0,3,1,35.0,0,0,8.0500,0.0,2,1,1,7,2,0,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,0.0,1,1,1,7,2,0,1,7
887,1,1,0,19.0,0,0,30.0000,0.0,0,2,0,1,2,1,1,1
888,0,3,0,NaN,1,2,23.4500,0.0,2,2,0,7,2,0,0,7
889,1,1,1,26.0,0,0,30.0000,1.0,0,1,1,2,0,1,1,2
